In [ ]:
import das.train 
from helper import RenderJSON
import helper, config_adapter
import pandas as pd, numpy as np,xarray as xr, yaml
from pathlib import Path
import scipy.io.wavfile

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
config_path = Path(params["config_path"])
config = config_adapter.load(config_path)
RenderJSON(config)

In [ ]:
fs, data = scipy.io.wavfile.read(params["audio_path"])
song = xr.Dataset()
song["data"] = xr.DataArray(data, dims="t")
song["t"] = np.arange(data.size)/fs
song["t"].attrs["fs"] = fs
song

In [ ]:
import das.utils
model_params = das.utils.load_params(str(Path(params["model_path"]).parent /"das"))
model_fs = model_params["samplerate_x_Hz"]
model_fs

In [ ]:
if model_fs != fs:
    interp = song.interp(t=np.arange(((song["t"].max() - song["t"].min())*model_fs).astype(int).item())/model_fs + song["t"].min())
    interp["t"].attrs["fs"] = model_fs
else:
    interp = song
interp

In [ ]:
events, segments, class_probabilities, class_names = das.predict.predict(interp["data"].values.reshape(-1, 1), 
                                                           model_save_name=str(Path(params["model_path"]).parent /"das"),
                                                           **config["das_predict_params"])
events, segments, class_probabilities, class_names

In [ ]:
segment_dims = np.where([val == "segment" for val in model_params["class_types"]])[0]
segment_names = [str(model_params["class_names"][segment_dim]) for segment_dim in segment_dims]
display(segment_names)
probas = xr.Dataset()
if len(interp["t"]) > class_probabilities.shape[0]:
  probas["proba"] = xr.DataArray(class_probabilities, dims=["t", "syb"]).pad(t=(0, len(interp["t"])-class_probabilities.shape[0]),constant_values=np.nan)
else:
  probas["proba"] = xr.DataArray(class_probabilities, dims=["t", "syb"])
probas["syb"] = segment_names

probas["t"] = interp["t"]
probas.to_netcdf("probas.nc")
probas

In [ ]:
import das.annot
df = das.annot.Events.from_predict(events, segments).to_df().sort_values("start_seconds").reset_index(drop=True).dropna(subset="start_seconds")
df

In [ ]:
df[df["name"]!="noise"].to_csv(params["out_annotations"])